In [1]:
import os
import re
from dotenv import load_dotenv
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
pd.set_option('display.max_columns', None)

# from typing import List, Any, Optional, Dict
# from rich.pretty import pprint

# def pretty_print(title: str=None, content: Any=None):
#     if title is None:
#         print(content)
#         return
#     print(title)
#     pprint(content)

from IPython.display import display, HTML, Markdown

In [2]:
from typing import Any, Optional, List
from rich.pretty import pprint
import textwrap

def pretty_print(title: Optional[str] = None, content: Any = None, exclude: Optional[List[str]] = None):
    """
    Prints the content prettily with text wrapping for better readability. If a title is provided, 
    it prints the title before the content. Can exclude specified attributes from being printed if 
    content is a dictionary.

    Args:
        title (Optional[str]): The title to print before the content. Defaults to None.
        content (Any): The content to be printed. Content will be wrapped for better readability.
        exclude (Optional[List[str]]): List of keys to exclude from the content if it's a dictionary. Defaults to None.
    """
    if exclude is not None and isinstance(content, dict):
        content = {k: v for k, v in content.items() if k not in exclude}
    
    if title is not None:
        print(title)
    
    if isinstance(content, str):
        content = textwrap.fill(content, width=80)
    
    pprint(content)

In [3]:
load_dotenv()
os.chdir(os.path.dirname(os.getcwd()))

In [4]:
df = pd.read_parquet("./data/splade_embeds.parquet")
# Convert column names to snake_case for compatibility with LanceDB
original_columns = df.columns
snake_case_columns = {col: re.sub(r'(?<!^)(?=[A-Z])', '_', col).lower() for col in original_columns}
df.rename(columns=snake_case_columns, inplace=True)
df.head(1)

,index,created_utc,full_link,id,body,title,text_label,flair_label,embeddings,token_count,llm_title,state,kmeans_label,topic_title,splade_embeddings
0,1078,1575952538,https://www.reddit.com/r/legaladvice/comments/...,e8lsen,I applied for a job and after two interviews I...,"Failed a drug test due to amphetamines, I have...",employment,5,"[9.475638042064453e-05, 0.0005111666301983955,...",493,"""Validity of Schedule II Drug Prescription in ...",PR,8,Employment Legal Concerns and Issues,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [5]:
from src.search.models import dataframe_to_text_nodes

nodes = dataframe_to_text_nodes(df=df, id_column='index', text_col='body', metadata_fields=['state', 'full_link'], embedding_column='embeddings')

In [6]:
pretty_print(nodes[1])

Node ID: 2098
Text: Hi everyone, thanks in advance for any guidance.  I was driving
home yesterday and managed to get pulled over outside Chattanooga
going 21 miles over the speed limit (76 in a 55).  It was unfortunate,
right at the part of the highway where the speed limit drops from 70
to 55 because of the mountains, and I just didn't adjust in time.  I
usually ...


None

In [7]:
from typing import Tuple
import numpy as np
from typing import cast
from llama_index.core.vector_stores.types import VectorStore
from llama_index.core.vector_stores import MetadataFilters, MetadataFilter
from llama_index.core.vector_stores import (
    VectorStoreQuery,
    VectorStoreQueryResult,
)
from llama_index.core.schema import BaseNode


def get_top_k_embeddings(
    query_embedding: List[float],
    doc_embeddings: List[List[float]],
    doc_ids: List[str],
    similarity_top_k: int = 5,
) -> Tuple[List[float], List]:
    """Get top nodes by similarity to the query."""
    # dimensions: D
    qembed_np = np.array(query_embedding)
    # dimensions: N x D
    dembed_np = np.array(doc_embeddings)
    # dimensions: N
    dproduct_arr = np.dot(dembed_np, qembed_np)
    # dimensions: N
    norm_arr = np.linalg.norm(qembed_np) * np.linalg.norm(
        dembed_np, axis=1, keepdims=False
    )
    # dimensions: N
    cos_sim_arr = dproduct_arr / norm_arr

    # now we have the N cosine similarities for each document
    # sort by top k cosine similarity, and return ids
    tups = [(cos_sim_arr[i], doc_ids[i]) for i in range(len(doc_ids))]
    sorted_tups = sorted(tups, key=lambda t: t[0], reverse=True)

    sorted_tups = sorted_tups[:similarity_top_k]

    result_similarities = [s for s, _ in sorted_tups]
    result_ids = [n for _, n in sorted_tups]
    return result_similarities, result_ids


def filter_nodes(nodes: List[BaseNode], filters: MetadataFilters):
    filtered_nodes = []
    for node in nodes:
        matches = True
        for f in filters.filters:
            if f.key not in node.metadata:
                matches = False
                continue
            if f.value != node.metadata[f.key]:
                matches = False
                continue
        if matches:
            filtered_nodes.append(node)
    return filtered_nodes


def dense_search(query: VectorStoreQuery, nodes: List[BaseNode]):
    """Dense search."""
    query_embedding = cast(List[float], query.query_embedding)
    doc_embeddings = [n.embedding for n in nodes]
    doc_ids = [n.node_id for n in nodes]
    return get_top_k_embeddings(
        query_embedding,
        doc_embeddings,
        doc_ids,
        similarity_top_k=query.similarity_top_k,
    )

In [8]:
from typing import Dict


class BaseVectorStore(VectorStore):
    """Simple custom Vector Store.

    Stores documents in a simple in-memory dict.

    """
    stores_text: bool = True
    
    def __init__(self) -> None:
        """Init params."""
        self.node_dict: Dict[str, BaseNode] = {}

    def get(self, text_id: str) -> List[float]:
        """Get embedding."""
        return self.node_dict[text_id]

    def add(
        self,
        nodes: List[BaseNode],
    ) -> List[str]:
        """Add nodes to index."""
        for node in nodes:
            self.node_dict[node.node_id] = node

    def delete(self, node_id: str, **delete_kwargs: Any) -> None:
        """
        Delete nodes using with node_id.

        Args:
            node_id: str

        """
        del self.node_dict[node_id]

    def query(
        self,
        query: VectorStoreQuery,
        **kwargs: Any,
    ) -> VectorStoreQueryResult:
        """Get nodes for response."""

        query_embedding = cast(List[float], query.query_embedding)
        doc_embeddings = [n.embedding for n in self.node_dict.values()]
        doc_ids = [n.node_id for n in self.node_dict.values()]

        similarities, node_ids = get_top_k_embeddings(
            query_embedding,
            doc_embeddings,
            doc_ids,
            similarity_top_k=query.similarity_top_k,
        )
        result_nodes = [self.node_dict[node_id] for node_id in node_ids]

        return VectorStoreQueryResult(
            nodes=result_nodes, similarities=similarities, ids=node_ids
        )

    def persist(self, persist_path, fs=None) -> None:
        """Persist the SimpleVectorStore to a directory.

        NOTE: we are not implementing this for now.

        """
        pass

In [9]:
vector_store = BaseVectorStore()
vector_store.add(nodes)

In [10]:
print(str(nodes[1]))

Node ID: 2098
Text: Hi everyone, thanks in advance for any guidance.  I was driving
home yesterday and managed to get pulled over outside Chattanooga
going 21 miles over the speed limit (76 in a 55).  It was unfortunate,
right at the part of the highway where the speed limit drops from 70
to 55 because of the mountains, and I just didn't adjust in time.  I
usually ...


In [11]:
import openai

client = openai.OpenAI()

In [12]:
query_str = "Can you tell me about medicinal marijuana use at work?"

query_embedding = client.embeddings.create(
                    input=query_str, model='text-embedding-ada-002'
                ).data[0].embedding

In [22]:
query_obj = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=5
)

query_result = vector_store.query(query_obj)
for similarity, node in zip(query_result.similarities, query_result.nodes):
    content = textwrap.fill(node.get_content(), width=150)
    print(
        f"[Node ID {node.node_id}] Similarity: {similarity}\n\n"
        f"{content}"
        "\n\n----------------\n\n"
    )

[Node ID 8075] Similarity: 0.8717243548229164

I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating
against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I
gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the
division of human rights at the labor department and they said I may or may not have a case.

----------------


[Node ID 7331] Similarity: 0.8698168024576537

Hello,  I was considering getting a medical marijuana card for my back pain. I've dealt with this chronically for years and it's one of the few things
that really help.  The problem is my company has a zero tolerance drug policy. The majority  of our employees are DOT regulated drivers so of course
they are not able to able to smoke marijuana even i

In [23]:
filters = MetadataFilters(
    filter_condition="or",
    filters=[
        MetadataFilter(key="state", value="MI", operator="text_match")
    ]
)

query_obj = VectorStoreQuery(
    query_embedding=query_embedding, similarity_top_k=3, filters=filters
)

query_result = vector_store.query(query_obj)
for similarity, node in zip(query_result.similarities, query_result.nodes):
    print(
        "\n----------------\n"
        f"[Node ID {node.node_id}] Similarity: {similarity}\n\n"
        f"{node.get_content(metadata_mode='all')}"
        "\n----------------\n\n"
    )


----------------
[Node ID 8075] Similarity: 0.8717243548229164

Metadata:
state = MI
full_link = https://www.reddit.com/r/legaladvice/comments/gr4q8s/is_it_legal_in_new_york_to_discriminate_against/
----------------------------------------
Content:
I'm a New York State medical marijuana patient. I also work in healthcare. I applied to a new job at a new hospital, and they are discriminating against me for being a medical marijuana patient. I was offered the job and accepted, but when I went to get my pre-employment physical conducted, I gave them my medical marijuana card and informed them that I am a patient. They are now refusing to hire me. Is this legal? I already contacted the division of human rights at the labor department and they said I may or may not have a case.
----------------



----------------
[Node ID 7331] Similarity: 0.8698168024576537

Metadata:
state = PA
full_link = https://www.reddit.com/r/legaladvice/comments/e8y7gr/considering_getting_a_medical_marijuana_card/

In [24]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_vector_store(vector_store)
query_engine = vector_index.as_query_engine()

response = query_engine.query("Write a detailed summary of the legal issues involving medical marijuana")
Markdown(str(response))

The legal issues involving medical marijuana can vary depending on the state. In some states, individuals who own a mobile home in a park may face restrictions on using medical marijuana on the park's property, even if they own their home outright. This can lead to conflicts between property rights and medical needs. Additionally, in the context of employment, discrimination against medical marijuana patients can occur, as seen in the case of a healthcare worker in New York who faced job discrimination after disclosing their medical marijuana patient status. This highlights the complex legal landscape surrounding medical marijuana use, where state laws, employment regulations, and individual rights intersect.

In [25]:
Markdown(response.source_nodes[0].get_content())

Asking this for a friend who lives in a mobile home park in California.  He wants to get a medical marijuana card for his medical conditions so he can get off his pain medicine, but the park is telling him that even though he owns his home 100%, because its on the park's land he can't use it or bring it on the property.  Can the park legally do this or is there some legal medical grounds for him?  

In [26]:
TEMPERATURE = 0.0
SIM_TOP_K = 6
RERANK_TOP_K = 3
WIN_SZ = 1024

import asyncio
from tqdm.asyncio import tqdm

import chromadb
from chromadb.api.models.Collection import Collection

from llama_index.core import (
    ServiceContext,
    SimpleDirectoryReader,
    StorageContext,
    VectorStoreIndex,
    get_response_synthesizer,
)
from llama_index.core import QueryBundle
from llama_index.llms.openai import OpenAI
from llama_index.core import PromptTemplate
from llama_index.core.llms.utils import LLMType
from llama_index.core.schema import NodeWithScore
from llama_index.core.indices.base import BaseIndex
from llama_index.core.retrievers import BaseRetriever
from llama_index.core.embeddings.utils import EmbedType
from llama_index.core.node_parser import SentenceSplitter
from llama_index.core.query_engine import RetrieverQueryEngine
from llama_index.legacy.vector_stores import ChromaVectorStore
from llama_index.core.base.llms.types import CompletionResponse
from llama_index.core.node_parser import SentenceWindowNodeParser

from llama_index.core.retrievers import BaseRetriever
from llama_index.core import get_response_synthesizer
from llama_index.legacy.core.response.schema import RESPONSE_TYPE
from llama_index.core.response_synthesizers import BaseSynthesizer
from llama_index.core.query_engine import CustomQueryEngine, BaseQueryEngine

In [27]:
li_model = OpenAI(model="gpt-4-1106-preview", temperature=TEMPERATURE)

In [28]:
vector_index = VectorStoreIndex.from_vector_store(
    vector_store=vector_store,
)

In [29]:
base_retriever = vector_index.as_retriever(similarity_top_k=20)

In [30]:
from collections import defaultdict


class MultiQueriesRetriever(BaseRetriever):
    def __init__(self, base_retriever: BaseRetriever, model:OpenAI):
        self.template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self._retrievers = [base_retriever]
        self.base_retriever = base_retriever
        self.model = model

    @classmethod
    def flatten(cls, lst: List[List[Any]]) -> List[Any]:
        return [element for sublist in lst for element in sublist]

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    async def run_gen_queries(self,generated_queries: List[str]) -> List[NodeWithScore]:
        tasks = list(map(lambda q: self.base_retriever.aretrieve(q), generated_queries))
        res = await tqdm.gather(*tasks)
        return MultiQueriesRetriever.flatten(res)

    def _retrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        return list()

    async def _aretrieve(self, query_bundle: QueryBundle) -> List[NodeWithScore]:
        query: str= query_bundle.query_str
        generated_queries: List[str] = self.gen_queries(query)
        pretty_print("generated_queries", generated_queries)
        node_with_scores = await self.run_gen_queries(generated_queries)
        # node_with_scores_uniqued = dict()
        # node_with_scores_uniqued = {node_with_score.get_content(): node_with_score for node_with_score in node_with_scores}
        # return node_with_scores_uniqued.values()
        # Fusion with ranking, ref:https://plg.uwaterloo.ca/~gvcormac/cormacksigir09-rrf.pdf
        output_documents = self._reciprocal_rank_fusion(node_with_scores)
        return output_documents
     
    def _reciprocal_rank_fusion(self, document_lists: List[List[NodeWithScore]]) -> List[NodeWithScore]:
        """
        Merge multiple lists of NodeWithScore and assign scores based on reciprocal rank fusion.
        
        Args:
            document_lists (List[List[NodeWithScore]]): A list of lists, where each sublist contains NodeWithScore objects.
        
        Returns:
            List[NodeWithScore]: A list of NodeWithScore objects with updated scores based on reciprocal rank fusion.
        
        The constant k is set to 61 (60 was suggested by the original paper,
        plus 1 as python lists are 0-based and the paper used 1-based ranking).
        """
        k = 61

        scores_map = defaultdict(int)
        documents_map = {}
        weights = self.weights if self.weights else [1 / len(document_lists)] * len(document_lists)

        # Calculate weighted reciprocal rank fusion score
        for documents, weight in zip(document_lists, weights):
            for rank, doc in enumerate(documents):
                scores_map[doc.id] += (weight * len(document_lists)) / (k + rank)
                documents_map[doc.id] = doc

        # Normalize scores. Note: len(results) / k is the maximum possible score,
        # achieved by being ranked first in all doc lists with non-zero weight.
        for id in scores_map:
            scores_map[id] /= len(document_lists) / k

        for doc in documents_map.values():
            doc.score = scores_map[doc.id]

        return documents_map.values()



In [31]:
mr = MultiQueriesRetriever(base_retriever, li_model)

In [32]:
import nest_asyncio
nest_asyncio.apply()

ls = mr.gen_queries("Is medical marijuana allowed at work in California?")
pretty_print("ls", ls)
rls = await mr.run_gen_queries(ls)

ls


[
│   'What are the regulations regarding medical marijuana use in the workplace in California?',
│   'Are there policies in place in California regarding the use of medical marijuana while on the job?',
│   'Can employees use medical marijuana at work in California?',
│   'What are the rules and guidelines for using medical marijuana in the workplace in California?',
│   'Is it permissible to bring medical marijuana to the workplace in California?'
]

100%|██████████| 5/5 [00:01<00:00,  3.60it/s]


In [33]:
len(rls)

100

In [35]:
rls[0].text

'Hello,\n\nI was considering getting a medical marijuana card for my back pain. I\'ve dealt with this chronically for years and it\'s one of the few things that really help.\n\nThe problem is my company has a zero tolerance drug policy. The majority  of our employees are DOT regulated drivers so of course they are not able to able to smoke marijuana even if they have a medical card, this makes sense to me.\n\nAs for the rest of us, even us office folk, we are all still subject to randoms.  Basically the logic is that everyone is considered to be in a "safety sensitive" position because we can drive on company time or drive a company vehicle. We have a memo in our drug and alcohol handbook to "all employees" stating that just because some states have legalized marijuana, does not mean we can disregard company policy. It does NOT say anything about medical cards.\n\nI live in Arizona where medical marijuana is legal. Is my company able to trump me having a medical card because of their p

In [36]:
class MultiQueriers:
    def __init__(self,
                 base_retriever: BaseRetriever,
                 base_query_engine: BaseQueryEngine,
                 model:OpenAI,
                 sub_queries_in_bundle_to_answer: bool=True):
        self.base_retriever = base_retriever
        self.base_query_engine = base_query_engine
        self.model = model
        self.sub_queries_in_bundle_to_answer = sub_queries_in_bundle_to_answer
        self.gen_q_template = PromptTemplate("""You are an AI language model assistant. Your task is to generate Five
    different versions of the given user question to retrieve relevant documents from a vector
    database. By generating multiple perspectives on the user question, your goal is to help
    the user overcome some of the limitations of the distance-based similarity search.
    Provide these alternative questions separated by newlines only.

    For example, these alternative questions:

    'What is Bill Gates known for?'
│   "Can you provide information about Bill Gates' background?"

    Not:

    '1. What is Bill Gates known for?'
│   "2. Can you provide information about Bill Gates' background?"

    Original question: {question}""")
        self.qa_prompt_template = PromptTemplate("""Here is the question you need to answer:

    \n --- \n {query_str} \n --- \n

    Here is any available background question + answer pairs:

    \n --- \n {q_a_pairs} \n --- \n

    Here is additional context relevant to the question:

    \n --- \n {context_str} \n --- \n

    Use the above context and any background question + answer pairs to answer the question: \n {query_str}
    """)

    def gen_queries(self, query: str) -> List[str]:
        gen_queries_model = OpenAI(model="gpt-3.5-turbo-0125", temperature=1.5)
        prompt = self.gen_q_template.format(question=query)
        res = gen_queries_model.complete(prompt)
        return res.text.split("\n")

    def query_by_retriever(self, query_str: str) ->str:
        nodes = self.base_retriever.retrieve(query_str)
        res = "\n\n".join([n.node.get_content() for n in nodes])
        return res

    async def run_gen_queries(self,generated_queries: List[str]) -> str:
        sub_query_qa_pairs = list()
        if self.sub_queries_in_bundle_to_answer:
            # Answer all queries in one bundle.
            tasks = list(map(lambda q: self.base_query_engine.aquery(q), generated_queries))
            res = await tqdm.gather(*tasks)
            for idx, (query, answer) in enumerate(zip(generated_queries, res)):
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            pretty_print("sub_query_qa_pairs", sub_query_qa_pairs)
            return "\n\n".join(sub_query_qa_pairs)
        else:
            # Answer queries in step-wise.
            # One sub-query will be answered based on the context of sub-query and
            # history of pairs of previous queries and answers.
            for idx, query in enumerate(generated_queries):
                pretty_print(f"{idx}. query", query)
                pretty_print(f"{idx}. sub_query_qa_pairs", sub_query_qa_pairs)
                context_str = self.query_by_retriever(query)
                sub_query = self.qa_prompt_template.format(
                                  query_str=query,
                                  q_a_pairs="\n\n".join(sub_query_qa_pairs),
                                  context_str=context_str
                )
                pretty_print("sub_query", sub_query)
                answer: str = self.model.complete(sub_query)
                qa_pair = f"Question {idx}: {query}\nAnswer: {answer}\n"
                sub_query_qa_pairs.append(qa_pair)
            return "\n\n".join(sub_query_qa_pairs)

    def query(self, query_str: str) -> CompletionResponse:
        return ""

    async def aquery(self, query_str: str) -> CompletionResponse:
        generated_queries: List[str] = self.gen_queries(query_str)
        sub_query_qa_pairs: str = await self.run_gen_queries(generated_queries)
        context_str = self.query_by_retriever(query_str)
        final_query: str = self.qa_prompt_template.format(query_str=query_str,
                                q_a_pairs=sub_query_qa_pairs,
                                context_str=context_str)
        pretty_print("final_query", final_query)
        response: str = self.model.complete(final_query)
        return response

In [37]:
import nest_asyncio
nest_asyncio.apply()

query_text = """What are the legal issues involving medical marijuana and employment?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in Markdown with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=True)
final_res = await mqe.aquery(query_text)
display(Markdown(final_res.text))

  0%|          | 0/6 [00:00<?, ?it/s]

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

In [38]:
query_text = """What are the legal issues involving medical marijuana and employment?

    The response should be presented as a list of key points, after creating the title of the content,
    formatted in Markdown with appropriate markup for clarity and organization.
    """
mqe=MultiQueriers(base_retriever,  vector_index.as_query_engine(), li_model, sub_queries_in_bundle_to_answer=False)
final_res = await mqe.aquery(query_text)
display(Markdown(final_res.text))

0. query


'What legal challenges are connected to the use of medical marijuana in the\nworkplace?'

0. sub_query_qa_pairs


[]

sub_query


...

sub_query


'Here is the question you need to answer:        ---   What are the legal rights\nand responsibilities regarding medical marijuana and employment?   ---\nHere is any available background question + answer pairs:        ---   Question\n0: What legal challenges are connected to the use of medical marijuana in the\nworkplace? Answer: The use of medical marijuana in the workplace presents\nseveral legal challenges, both for employees and employers. These challenges\narise from the conflict between state laws that may permit medical marijuana use\nand federal laws that classify marijuana as an illegal substance. Here are some\nof the key legal issues connected to the use of medical marijuana in the\nworkplace:  1. **Federal vs. State Law Conflict**: Marijuana remains illegal\nunder federal law as a Schedule I controlled substance. This means that even if\nan employee is legally prescribed marijuana for medical purposes under state\nlaw, they are still violating federal law. Employers who are subject to federal\nregulations, especially those with federal contracts or those in industries like\ntransportation that are regulated by the Department of Transportation (DOT), may\nbe required to enforce a drug-free workplace policy that prohibits marijuana\nuse.  2. **Disability Discrimination**: Employees who use medical marijuana may\nhave underlying health conditions that qualify as disabilities under the\nAmericans with Disabilities Act (ADA) or similar state laws. While the ADA does\nnot protect illegal drug use, employees may argue that adverse actions taken\nagainst them because of their medical marijuana use are actually based on their\nunderlying disability, which could lead to claims of discrimination.  3.\n**Reasonable Accommodation**: Employers are generally required to provide\nreasonable accommodation for employees with disabilities. However, courts have\nbeen divided on whether allowing the use of medical marijuana is a reasonable\naccommodation, especially since it is illegal under federal law.  4. **Drug\nTesting Policies**: Employers with drug testing policies may face challenges\nwhen an employee who is a medical marijuana patient tests positive for THC. The\nlegality of taking adverse employment action against such an employee can vary\nby state, and some states have protections for medical marijuana users.  5.\n**Safety-Sensitive Positions**: Employers have a duty to maintain a safe\nworkplace. For positions that are safety-sensitive, employers may argue that the\nuse of medical marijuana poses a risk to safety and, therefore, must be\nprohibited regardless of an employee\'s medical marijuana patient status.  6.\n**Privacy Concerns**: Disclosing medical marijuana use may raise privacy issues,\nespecially if an employer discloses an employee\'s medical condition or\nprescription drug use to others in the workplace. There may also be concerns\nabout whether such disclosures violate the Health Insurance Portability and\nAccountability Act (HIPAA).  7. **Zero Tolerance Policies**: Employers with\nzero-tolerance drug policies may face legal challenges if they do not modify\ntheir policies to account for state laws that protect medical marijuana users.\nEmployers must balance their drug-free workplace policies with the rights of\nemployees under state law.  8. **Workers\' Compensation and Unemployment\nBenefits**: There may be legal questions about whether an employee who uses\nmedical marijuana is eligible for workers\' compensation or unemployment benefits\nif they are injured on the job or terminated for marijuana use.  9. **Housing\nand Land Use**: For employees who live in employer-provided housing or in\ncertain communities, there may be rules or lease agreements that prohibit the\nuse of marijuana on the property, which can affect the employee\'s ability to use\nmedical marijuana.  10. **Employment Contracts and Collective Bargaining\nAgreements**: Some employment contracts or collective bargaining agreements may\nhave

final_query


'Here is the question you need to answer:        ---   What are the legal issues\ninvolving medical marijuana and employment?      The response should be\npresented as a list of key points, after creating the title of the content,\nformatted in Markdown with appropriate markup for clarity and organization.\n---        Here is any available background question + answer pairs:        ---\nQuestion 0: What legal challenges are connected to the use of medical marijuana\nin the workplace? Answer: The use of medical marijuana in the workplace presents\nseveral legal challenges, both for employees and employers. These challenges\narise from the conflict between state laws that may permit medical marijuana use\nand federal laws that classify marijuana as an illegal substance. Here are some\nof the key legal issues connected to the use of medical marijuana in the\nworkplace:  1. **Federal vs. State Law Conflict**: Marijuana remains illegal\nunder federal law as a Schedule I controlled substance. This means that even if\nan employee is legally prescribed marijuana for medical purposes under state\nlaw, they are still violating federal law. Employers who are subject to federal\nregulations, especially those with federal contracts or those in industries like\ntransportation that are regulated by the Department of Transportation (DOT), may\nbe required to enforce a drug-free workplace policy that prohibits marijuana\nuse.  2. **Disability Discrimination**: Employees who use medical marijuana may\nhave underlying health conditions that qualify as disabilities under the\nAmericans with Disabilities Act (ADA) or similar state laws. While the ADA does\nnot protect illegal drug use, employees may argue that adverse actions taken\nagainst them because of their medical marijuana use are actually based on their\nunderlying disability, which could lead to claims of discrimination.  3.\n**Reasonable Accommodation**: Employers are generally required to provide\nreasonable accommodation for employees with disabilities. However, courts have\nbeen divided on whether allowing the use of medical marijuana is a reasonable\naccommodation, especially since it is illegal under federal law.  4. **Drug\nTesting Policies**: Employers with drug testing policies may face challenges\nwhen an employee who is a medical marijuana patient tests positive for THC. The\nlegality of taking adverse employment action against such an employee can vary\nby state, and some states have protections for medical marijuana users.  5.\n**Safety-Sensitive Positions**: Employers have a duty to maintain a safe\nworkplace. For positions that are safety-sensitive, employers may argue that the\nuse of medical marijuana poses a risk to safety and, therefore, must be\nprohibited regardless of an employee\'s medical marijuana patient status.  6.\n**Privacy Concerns**: Disclosing medical marijuana use may raise privacy issues,\nespecially if an employer discloses an employee\'s medical condition or\nprescription drug use to others in the workplace. There may also be concerns\nabout whether such disclosures violate the Health Insurance Portability and\nAccountability Act (HIPAA).  7. **Zero Tolerance Policies**: Employers with\nzero-tolerance drug policies may face legal challenges if they do not modify\ntheir policies to account for state laws that protect medical marijuana users.\nEmployers must balance their drug-free workplace policies with the rights of\nemployees under state law.  8. **Workers\' Compensation and Unemployment\nBenefits**: There may be legal questions about whether an employee who uses\nmedical marijuana is eligible for workers\' compensation or unemployment benefits\nif they are injured on the job or terminated for marijuana use.  9. **Housing\nand Land Use**: For employees who live in employer-provided housing or in\ncertain communities, there may be rules or lease agreements that prohibit the\nuse of marijuana on the property, which can affect the employee\'s ability to 

# Key Legal Issues Involving Medical Marijuana and Employment

The intersection of medical marijuana legalization and employment law creates a complex legal landscape for both employers and employees. Below are key points outlining the legal issues that arise in this context:

1. **Federal vs. State Law Conflict**: Marijuana remains a Schedule I controlled substance under federal law, which conflicts with state laws that have legalized medical marijuana. This dichotomy creates challenges for employers who must comply with federal regulations and for employees who are legally prescribed marijuana under state law.

2. **Disability Discrimination**: Employees using medical marijuana may have underlying health conditions that qualify as disabilities. While the Americans with Disabilities Act (ADA) does not protect illegal drug use, there is a gray area when it comes to state-legal medical marijuana and potential discrimination claims.

3. **Reasonable Accommodation**: Employers are required to provide reasonable accommodation for employees with disabilities, but it is unclear if medical marijuana use falls under this requirement, especially given its federal illegality.

4. **Drug Testing Policies**: Employers with drug testing policies face challenges with medical marijuana patients who test positive for THC. State laws vary on whether employers can take adverse action based on a positive marijuana test.

5. **Safety-Sensitive Positions**: Employers have a duty to maintain a safe workplace, which can lead to prohibiting medical marijuana use in safety-sensitive positions, even if the employee is a legal medical marijuana patient.

6. **Privacy Concerns**: Disclosing medical marijuana use may raise privacy issues, potentially violating laws like the Health Insurance Portability and Accountability Act (HIPAA) if an employee's medical condition or prescription drug use is disclosed improperly.

7. **Zero Tolerance Policies**: Employers with zero-tolerance drug policies may face legal challenges if they do not accommodate state laws protecting medical marijuana users, requiring a balance between drug-free workplace policies and employee rights.

8. **Workers' Compensation and Unemployment Benefits**: Legal questions arise regarding the eligibility of medical marijuana users for workers' compensation or unemployment benefits if they are injured on the job or terminated for marijuana use.

9. **Housing and Land Use**: Employees living in employer-provided housing or certain communities may face restrictions on marijuana use on the property, affecting their ability to use medical marijuana.

10. **Employment Contracts and Collective Bargaining Agreements**: Specific provisions in employment contracts or collective bargaining agreements may impact medical marijuana users differently.

11. **Legal Uncertainty and Litigation**: The evolving legal landscape around medical marijuana use leads to uncertainty and potential litigation as parties seek to clarify their rights and obligations under the law.

12. **Employer Accommodation Policies**: Employers may need to review and revise their accommodation policies to address medical marijuana use, especially in states with explicit protections for medical marijuana patients.

In summary, employers and employees must navigate a patchwork of state protections, federal prohibitions, and evolving case law to understand their rights and obligations regarding medical marijuana in the workplace. Legal counsel is often necessary to address specific situations.

In [39]:
display(Markdown(final_res.text))

# Key Legal Issues Involving Medical Marijuana and Employment

The intersection of medical marijuana legalization and employment law creates a complex legal landscape for both employers and employees. Below are key points outlining the legal issues that arise in this context:

1. **Federal vs. State Law Conflict**: Marijuana remains a Schedule I controlled substance under federal law, which conflicts with state laws that have legalized medical marijuana. This dichotomy creates challenges for employers who must comply with federal regulations and for employees who are legally prescribed marijuana under state law.

2. **Disability Discrimination**: Employees using medical marijuana may have underlying health conditions that qualify as disabilities. While the Americans with Disabilities Act (ADA) does not protect illegal drug use, there is a gray area when it comes to state-legal medical marijuana and potential discrimination claims.

3. **Reasonable Accommodation**: Employers are required to provide reasonable accommodation for employees with disabilities, but it is unclear if medical marijuana use falls under this requirement, especially given its federal illegality.

4. **Drug Testing Policies**: Employers with drug testing policies face challenges with medical marijuana patients who test positive for THC. State laws vary on whether employers can take adverse action based on a positive marijuana test.

5. **Safety-Sensitive Positions**: Employers have a duty to maintain a safe workplace, which can lead to prohibiting medical marijuana use in safety-sensitive positions, even if the employee is a legal medical marijuana patient.

6. **Privacy Concerns**: Disclosing medical marijuana use may raise privacy issues, potentially violating laws like the Health Insurance Portability and Accountability Act (HIPAA) if an employee's medical condition or prescription drug use is disclosed improperly.

7. **Zero Tolerance Policies**: Employers with zero-tolerance drug policies may face legal challenges if they do not accommodate state laws protecting medical marijuana users, requiring a balance between drug-free workplace policies and employee rights.

8. **Workers' Compensation and Unemployment Benefits**: Legal questions arise regarding the eligibility of medical marijuana users for workers' compensation or unemployment benefits if they are injured on the job or terminated for marijuana use.

9. **Housing and Land Use**: Employees living in employer-provided housing or certain communities may face restrictions on marijuana use on the property, affecting their ability to use medical marijuana.

10. **Employment Contracts and Collective Bargaining Agreements**: Specific provisions in employment contracts or collective bargaining agreements may impact medical marijuana users differently.

11. **Legal Uncertainty and Litigation**: The evolving legal landscape around medical marijuana use leads to uncertainty and potential litigation as parties seek to clarify their rights and obligations under the law.

12. **Employer Accommodation Policies**: Employers may need to review and revise their accommodation policies to address medical marijuana use, especially in states with explicit protections for medical marijuana patients.

In summary, employers and employees must navigate a patchwork of state protections, federal prohibitions, and evolving case law to understand their rights and obligations regarding medical marijuana in the workplace. Legal counsel is often necessary to address specific situations.